In [9]:
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from time import time
import os

In [6]:
draft_data_path = '../data/nhl-draft-picks-2009-2018/'
os.listdir(draft_data_path)

['2009.csv',
 '2010.csv',
 '2011.csv',
 '2012.csv',
 '2013.csv',
 '2014.csv',
 '2015.csv',
 '2016.csv',
 '2017.csv',
 '2018.csv']

In [11]:
t = time()
# glob all .csv files from NHL Draft data
pattern = '*.csv'
csv_files = glob(draft_data_path + pattern)

#  Iterate over csv_files
frames = []
for csv in csv_files:
    #  Read csv into a DataFrame: df
    df = pd.read_csv(csv)
    df['year'] = csv[-8:-5]
    # Append df to frames
    frames.append(df)
df = pd.concat(frames)
elapsed = time() - t
print("----- DataFrame with NHL Draft Data loaded"
      "\nin {0:.2f} seconds".format(elapsed) + 
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(df.shape[0], df.shape[1]) + 
      "\n-- Column names:\n", df.columns)

----- DataFrame with NHL Draft Data loaded
in 0.19 seconds
with 2,119 rows
and 21 columns
-- Column names:
 Index(['Overall', 'Team', 'Player', 'Nat.', 'Pos', 'Age', 'To', 'Amateur Team',
       'GP', 'G', 'A', 'PTS', '+/-', 'PIM', 'GP.1', 'W', 'L', 'T/O', 'SV%',
       'GAA', 'year'],
      dtype='object')


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2119 entries, 0 to 216
Data columns (total 21 columns):
Overall         2119 non-null int64
Team            2119 non-null object
Player          2119 non-null object
Nat.            2118 non-null object
Pos             2118 non-null object
Age             2118 non-null float64
To              728 non-null float64
Amateur Team    2118 non-null object
GP              728 non-null float64
G               728 non-null float64
A               728 non-null float64
PTS             728 non-null float64
+/-             724 non-null float64
PIM             728 non-null float64
GP.1            55 non-null float64
W               54 non-null float64
L               54 non-null float64
T/O             54 non-null float64
SV%             55 non-null float64
GAA             55 non-null float64
year            2119 non-null object
dtypes: float64(14), int64(1), object(6)
memory usage: 444.2+ KB
